In [ ]:
import os
import ipywidgets as widgets
from IPython.display import display, clear_output
from Corpus import Corpus
from SearchEngine import SearchEngine
import praw

state = {'corpus': None, 'engine': None}

inputKeyword = widgets.Text(placeholder='Sujet du corpus', description='Sujet :')
loadButton = widgets.Button(description="Charger", button_style='primary')
loadOut = widgets.Output()

def onLoad(b):
    with loadOut:
        clear_output()
        val = inputKeyword.value.strip()
        if not val: 
            print("Entrez un mot-clé")
            return
        
        csv_dir = "csv"
        os.makedirs(csv_dir, exist_ok=True)
        filename = os.path.join(csv_dir, f"{val}_corpus.csv")

        c = Corpus(val)
        c.id2doc = {}
        c.authors = {}
        c.ndoc = 0
        c.naut = 0
        c.nom = val
        state['corpus'] = c
        
        if os.path.exists(filename):
            state['corpus'].load(filename)
            print(f"Importé depuis {filename}")
        else:
            reddit = praw.Reddit(client_id='CvBa0giChE03eJkVvvb_1A',
                                 client_secret='hIaWPpSE821dHejacJKpeHJwr_g3SQ',
                                 user_agent='PrawTest')
            state['corpus'].loadCorpus(reddit, val)
            state['corpus'].save(filename)
            print("Nouveau corpus créé")
            
        state['engine'] = SearchEngine(state['corpus'])

loadButton.on_click(onLoad)
display(widgets.HBox([inputKeyword, loadButton]), loadOut)

inputSearch = widgets.Text(description='Chercher :')
inputAuthor = widgets.Text(description='Auteur :')
filterType = widgets.Dropdown(options=['Tous', 'Reddit', 'Arxiv'], description='Source :')
slider = widgets.IntSlider(value=50, min=1, max=100, description='Top :')
searchButton = widgets.Button(description="Rechercher", button_style='primary')
resOut = widgets.Output()

def onSearch(b):
    with resOut:
        clear_output()
        if not state['engine']: 
            print("Chargez un corpus")
            return
        
        res = state['engine'].search(inputSearch.value, nResults=slider.value)
        
        if inputAuthor.value:
            res = res[res['auteur'].str.contains(inputAuthor.value, case=False, na=False)]
        if filterType.value != 'Tous':
            res = res[res['titre'].apply(lambda t: any(d.titre == t and d.type == filterType.value for d in state['corpus'].id2doc.values()))]
        
        if not res.empty:
            display(res[['score','titre','auteur','date']])
        else:
            print("Rien trouvé")

searchButton.on_click(onSearch)

statsButton = widgets.Button(description="Stats", button_style='info')
statsOut = widgets.Output()

def showStats(b):
    with statsOut:
        clear_output()
        if state['corpus']:
            print(state['corpus'])
            state['corpus'].afficherAuteurs()

statsButton.on_click(showStats)

ui_top = widgets.VBox([
    widgets.HBox([inputSearch, inputAuthor]),
    widgets.HBox([filterType, slider])
])

ui_buttons = widgets.HBox([searchButton, statsButton])
display(ui_top, ui_buttons, resOut, statsOut)


Output()

Output()

Output()